In [2]:
import pandas as pd
from config import api_key
import requests
import os
import math
from datetime import datetime
from config import finn
import finnhub
finnhub_client = finnhub.Client(api_key=finn)
import time
import datetime
now = datetime.datetime.now()

In [3]:
# url for api key
option_url = 'https://api.tdameritrade.com/v1/marketdata/chains'


In [ ]:
#url for td
url = "https://api.tdameritrade.com/v1/marketdata/{}/pricehistory".format(symbol)

In [3]:
symbol = 'SPY'

In [4]:
#parameters
paylod = {'apikey': api_key,
      'frequencyType': 'daily',
      'frequency': '1',
      'period' : '1',
      'periodType' : 'month',
      'needExtendedHoursData': 'False'
     }

In [29]:

#requesting data
content = requests.get(url = url, params=paylod)

data = content.json()

open_tick = []
high_tick = []
low_tick = []
close_tick = []
volume_tick = []
datetime_tick = []

for i in range(len(data["candles"])):
    open_tick.append(data["candles"][i]["open"])
    high_tick.append(data["candles"][i]["high"])
    low_tick.append(data["candles"][i]["low"])
    close_tick.append(data["candles"][i]["close"])
    volume_tick.append(data["candles"][i]["volume"])
    datetime_tick.append(data["candles"][i]["datetime"])
    
close = close_tick[-1]

In [11]:
# input conditions for options
symbol = input("Ticker")
strikecount = input("Strikecount")
expMonth = input('Exp Month')
contract = input('Contract type PUT, CALL, ALL')
strike = input("Strike Price")

Ticker
Strikecount20
Exp MonthFEB
Contract type PUT, CALL, ALLALL
Strike Price


In [36]:
test = int(time.time())
info = finnhub_client.stock_candles(symbol, 'D', test, test)

test = int(time.time())
info = finnhub_client.stock_candles(symbol, 'D', test, test)

if info["s"] == 'no_data':
    info = finnhub_client.stock_candles(symbol, 'D', test, test)
    
    #if info["s"] == 'no_data':
        #continue

data  = pd.DataFrame(info, index= [0])
data.columns = ["Close", "High", "Low", "Open", "S", "Timestamp", "Volume"]
data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
data = data.drop(columns=["S", "Timestamp"])
data = data.set_index("Datetime")

close = data['Close'].values[0]

/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_37413/2975534421.py:15: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])


In [32]:
#uses api key to extract data
params = {'apikey':api_key,
         'symbol':symbol,
         'contractType': contract,
          'strikeCount': strikecount,
          'includeQuotes': False,
          'stratgey': 'SINGLE',
          'expMonth': expMonth,
          'optionType': 'S'
          
         
         }

content = requests.get(url = option_url, params=params)

data = content.json()


{'symbol': 'SPY',
 'status': 'SUCCESS',
 'underlying': None,
 'strategy': 'SINGLE',
 'interval': 0.0,
 'isDelayed': True,
 'isIndex': False,
 'interestRate': 4.816,
 'underlyingPrice': 396.235,
 'volatility': 29.0,
 'daysToExpiration': 0.0,
 'numberOfContracts': 240,
 'putExpDateMap': {'2023-02-01:10': {'386.0': [{'putCall': 'PUT',
     'symbol': 'SPY_020123P386',
     'description': 'SPY Feb 1 2023 386 Put (Weekly)',
     'exchangeName': 'OPR',
     'bid': 1.78,
     'ask': 1.81,
     'last': 1.79,
     'mark': 1.8,
     'bidSize': 160,
     'askSize': 191,
     'bidAskSize': '160X191',
     'lastSize': 0,
     'highPrice': 3.99,
     'lowPrice': 1.74,
     'openPrice': 0.0,
     'closePrice': 1.74,
     'totalVolume': 1147,
     'tradeDate': None,
     'tradeTimeInLong': 1674249083384,
     'quoteTimeInLong': 1674249299950,
     'netChange': 0.05,
     'volatility': 18.613,
     'delta': -0.218,
     'gamma': 0.021,
     'theta': -0.151,
     'vega': 0.217,
     'rho': -0.03,
     'o

In [12]:
#iterate through dates in data
for i in data["putExpDateMap"]:
    
    #create outline for 
    df = {}
    date = i[:10]
    volume = []
    theta = []
    gamma = []
    vega = []
    delta = []
    interest = []
    time = []
    
    #iterate through each strike
    for strike in data["putExpDateMap"][i]:
        df[strike] = []
        df[strike].append(data["putExpDateMap"][i][strike][0]["totalVolume"])
        df[strike].append(data["putExpDateMap"][i][strike][0]["delta"])
        df[strike].append(data["putExpDateMap"][i][strike][0]["theta"])
        df[strike].append(data["putExpDateMap"][i][strike][0]["gamma"])
        df[strike].append(data["putExpDateMap"][i][strike][0]["vega"])
        df[strike].append(data["putExpDateMap"][i][strike][0]["openInterest"])
        df[strike].append(data["putExpDateMap"][i][strike][0]["timeValue"])
        df[strike].append(data["putExpDateMap"][i][strike][0]["closePrice"])
    
    #convert to dataframe
    column = ["Volume", "Delta", "Theta", "Gamma", "Vega", "OI", "Time Value",'Last']
    option_df = pd.DataFrame.from_dict(df, orient='index', columns = column)
    
    #convert to csv
    mypath = f"Options/{symbol}"
    if not os.path.isdir(mypath):
        os.makedirs(mypath)

    upload_file = f"{mypath}/{date}.csv"
    option_df.to_csv(upload_file)
        

# Put Call Parity

In [4]:
# Dictionary for months 

months_ref = {1: 31,
          2: 28,
          3: 31,
          4: 30,
          5: 31,
          6: 30,
          7: 31,
          8: 31,
          9: 30,
          10: 31,
          11: 30,
          12: 31,
   }

In [5]:
# Tickers for S&P 500 stocks
tickers = pd.read_csv("S&P.csv")["S&P"].tolist()

In [9]:
# Tickers for all optionable stocks

tickers = pd.read_csv('Stocks.csv')['Tickers'].tolist()

In [12]:
# Using last price

arb = {}

for symbol in tickers:
    
    # obtain recent close for stock
    
    test = int(time.time())
    info = finnhub_client.stock_candles(symbol, 'D', test, test)

    test = int(time.time())
    info = finnhub_client.stock_candles(symbol, 'D', test, test)

    while info["s"] == 'no_data':
        time.sleep(1)
        
        info = finnhub_client.stock_candles(symbol, 'D', test, test)

    data  = pd.DataFrame(info, index= [0])
    data.columns = ["Close", "High", "Low", "Open", "S", "Timestamp", "Volume"]
    data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
    data = data.drop(columns=["S", "Timestamp"])
    data = data.set_index("Datetime")

    close = data['Close'].values[0]
    

    #uses api key to extract options data
    params = {'apikey':api_key,
             'symbol':symbol,
             'contractType': contract,
              'strikeCount': strikecount,
              'includeQuotes': False,
              'stratgey': 'SINGLE',
              'expMonth': expMonth,
              'optionType': 'S'


             }

    content = requests.get(url = option_url, params=params)

    data = content.json()


    

    dates_ref = []

    int_year = now.year
    int_month = now.month
    int_day = now.day

    for i in data['putExpDateMap']:
        dates_ref.append(i)

    for i in dates_ref:

        #create outline for 
        df = {}
        date = i[:10]
        volume = []
        theta = []
        gamma = []
        vega = []
        delta = []
        interest = []

        #iterate through each strike
        for strike in data["putExpDateMap"][i]:
            df[strike] = {'Put': '', 'Call':''}
            df[strike]['Put'] = data["putExpDateMap"][i][strike][0]["closePrice"]
            df[strike]['Put - Volume'] = data["putExpDateMap"][i][strike][0]["totalVolume"]

        for strike in data["callExpDateMap"][i]:

            df[strike]['Call'] = data["callExpDateMap"][i][strike][0]["closePrice"]
            df[strike]['Call - Volume'] = data["callExpDateMap"][i][strike][0]["totalVolume"]



        label = i

        strike_date = i.split('-')

        year = int(strike_date[0])

        month = int(strike_date[1])

        day = int(strike_date[2].split(':')[0])

        # amount of days until expiratoin
        
        count = int(strike_date[2].split(':')[1]) / 365
            
        #convert to dataframe
        option_df = pd.DataFrame.from_dict(df, orient='index')
        
        # iterate through df
        
        for row, i in option_df.iterrows():

            
            # calculates profit through put call parity equation
        
            C = i['Call']

            P = i['Put']

            left_side = close + P

            right_side = C + float(strike) * math.exp(-0.04*count)
            
            # if left side is greater than right side by 5 dollars

            if (left_side - right_side) > 5:
                
                C = i['Call - ask']

                P = i['Put - bid']

                left_side = close + P

                right_side = C + float(strike) * math.exp(-0.04*count)

                arb[f'{label} - {strike}'] = {'Put': df[strike]['Put - bid'], 'Call': df[strike]['Call - ask'], 'Right Side' : right_side, 'Left Side': left_side, 'Symbol': symbol, 'Put - Volume': df[strike]['Put - Volume'], 'Call - Volume': df[strike]['Call - Volume'], 'Price': close}

             # if right side is greater than left side by 5 dollars
            if (right_side - left_side) > 5:
                
                C = i['Call - bid']

                P = i['Put - ask']
                
                left_side = close + P

                right_side = C + float(strike) * math.exp(-0.04*count)

                arb[f'{label} - {strike}'] = {'Put': df[strike]['Put - ask'], 'Call': df[strike]['Call - bid'], 'Right Side' : right_side, 'Left Side': left_side, 'Symbol': symbol, 'Call - Volume': df[strike]['Call - Volume'], 'Put - Volume': df[strike]['Put - Volume'], 'Price': close}

        
                
        #convert to csv
        mypath = f"Put_Call_Polarity/{symbol}"
        if not os.path.isdir(mypath):
            os.makedirs(mypath)

        upload_file = f"{mypath}/{date}.csv"
        option_df.to_csv(upload_file)
        
    time.sleep(3)


/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_47262/2537410017.py:20: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_47262/2537410017.py:20: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_47262/2537410017.py:20: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
/var/folders/

/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_47262/2537410017.py:20: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_47262/2537410017.py:20: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_47262/2537410017.py:20: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
/var/folders/

KeyboardInterrupt: 

In [70]:
   # obtain recent close for stock
    
    test = int(time.time())
    info = finnhub_client.stock_candles(symbol, 'D', test, test)

    test = int(time.time())
    info = finnhub_client.stock_candles(symbol, 'D', test, test)

    while info["s"] == 'no_data':
        time.sleep(1)
        
        info = finnhub_client.stock_candles(symbol, 'D', test, test)

    data  = pd.DataFrame(info, index= [0])
    data.columns = ["Close", "High", "Low", "Open", "S", "Timestamp", "Volume"]
    data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
    data = data.drop(columns=["S", "Timestamp"])
    data = data.set_index("Datetime")

    close = data['Close'].values[0]
    

    #uses api key to extract options data
    params = {'apikey':api_key,
             'symbol':symbol,
             'contractType': contract,
              'strikeCount': strikecount,
              'includeQuotes': False,
              'stratgey': 'SINGLE',
              'expMonth': expMonth,
              'optionType': 'S'


             }

    content = requests.get(url = option_url, params=params)

    data = content.json()


    

    dates_ref = []

    int_year = now.year
    int_month = now.month
    int_day = now.day

    for i in data['putExpDateMap']:
        dates_ref.append(i)

    for i in dates_ref:

        #create outline for 
        df = {}
        date = i[:10]
        volume = []
        theta = []
        gamma = []
        vega = []
        delta = []
        interest = []

        #iterate through each strike
        for strike in data["putExpDateMap"][i]:
            df[strike] = {'Put': '', 'Call':''}
            df[strike]['Put'] = data["putExpDateMap"][i][strike][0]["closePrice"]
            df[strike]['Put - Volume'] = data["putExpDateMap"][i][strike][0]["totalVolume"]

        for strike in data["callExpDateMap"][i]:

            df[strike]['Call'] = data["callExpDateMap"][i][strike][0]["closePrice"]
            df[strike]['Call - Volume'] = data["callExpDateMap"][i][strike][0]["totalVolume"]



        label = i

        strike_date = i.split('-')

        year = int(strike_date[0])

        month = int(strike_date[1])

        day = int(strike_date[2].split(':')[0])
        
        # amount of days until expiratoin
        
        count = int(strike_date[2].split(':')[1]) / 365
            
        #convert to dataframe
        option_df = pd.DataFrame.from_dict(df, orient='index')
        
        # iterate through df
        
        for row, i in option_df.iterrows():

            
            # calculates profit through put call parity equation
        
            C = i['Call']

            P = i['Put']

            left_side = close + P

            right_side = C + float(strike) * math.exp(-0.04*count)
            
            # if left side is greater than right side by 5 dollars

            if (left_side - right_side) > 5:
                
                C = i['Call - ask']

                P = i['Put - bid']

                left_side = close + P

                right_side = C + float(strike) * math.exp(-0.04*count)

                arb[f'{label} - {strike}'] = {'Put': df[strike]['Put - bid'], 'Call': df[strike]['Call - ask'], 'Right Side' : right_side, 'Left Side': left_side, 'Symbol': symbol, 'Put - Volume': df[strike]['Put - Volume'], 'Call - Volume': df[strike]['Call - Volume'], 'Price': close}

             # if right side is greater than left side by 5 dollars
            if (right_side - left_side) > 5:
                
                C = i['Call - bid']

                P = i['Put - ask']
                
                left_side = close + P

                right_side = C + float(strike) * math.exp(-0.04*count)

                arb[f'{label} - {strike}'] = {'Put': df[strike]['Put - ask'], 'Call': df[strike]['Call - bid'], 'Right Side' : right_side, 'Left Side': left_side, 'Symbol': symbol, 'Call - Volume': df[strike]['Call - Volume'], 'Put - Volume': df[strike]['Put - Volume'], 'Price': close}

        
                
        #convert to csv
        mypath = f"Put_Call_Polarity/{symbol}"
        if not os.path.isdir(mypath):
            os.makedirs(mypath)

        upload_file = f"{mypath}/{date}.csv"
        option_df.to_csv(upload_file)
        
    time.sleep(3)


/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_37413/1562405913.py:20: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_37413/1562405913.py:20: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_37413/1562405913.py:20: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
/var/folders/

KeyboardInterrupt: 

In [71]:
# dataframe for all profitable put call parity trades using the equation

pd.DataFrame.from_dict(arb, orient='index')

,Put,Call,Right Side,Left Side,Symbol,Call - Volume,Put - Volume,Price
2023-02-03:11 - 152.5,12.90,0.45,154.956275,146.51,AAPL,3063,203,141.11
2023-02-10:18 - 152.5,13.10,0.64,154.879475,147.61,AAPL,664,22,141.11
2023-02-17:25 - 152.5,13.30,0.92,153.002764,154.41,AAPL,1527,7,141.11
2023-02-24:32 - 152.5,13.60,0.98,155.816142,147.16,AAPL,437,2,141.11
2023-02-03:11 - 267.5,26.05,0.49,268.527729,261.88,MSFT,359,0,242.58
2023-02-10:18 - 270.0,29.80,0.62,270.507922,264.38,MSFT,265,0,242.58
2023-02-17:25 - 270.0,28.75,0.74,270.881286,264.73,MSFT,2172,2,242.58
2023-02-24:32 - 290.0,48.35,0.12,289.324795,281.23,MSFT,10,0,242.58
2023-02-03:11 - 107.0,10.80,1.23,109.811091,103.92,AMZN,530,270,97.52
2023-02-10:18 - 107.0,11.00,1.53,110.089140,104.27,AMZN,79,0,97.52


In [13]:
pd.DataFrame.from_dict(arb, orient='index')

,Put,Call,Right Side,Left Side,Symbol,Call - Volume,Put - Volume,Price
2023-02-17:24 - 200.0,40.94,0.04,199.534664,191.6300,A,0,0,155.6900
2023-02-03:10 - 62.0,10.27,0.05,62.622092,56.7500,AA,1,0,52.8700
2023-02-10:17 - 62.0,10.31,0.13,62.924601,57.0800,AA,0,0,52.8700
2023-02-17:24 - 70.0,25.19,0.00,69.816132,60.3800,ABM,0,0,45.1900
2023-02-24:31 - 62.0,10.50,0.36,63.509727,57.7100,AA,4,2,52.8700
2023-02-17:24 - 15.0,6.05,0.00,15.010600,9.9700,ACEL,0,0,8.8900
2023-02-17:24 - 25.0,10.60,0.00,24.934333,17.4600,ABR,0,0,14.3600
2023-02-17:24 - 7.5,5.10,0.00,7.780300,2.7600,ACER,0,0,2.3700
2023-02-03:10 - 22.0,5.83,0.01,22.265904,17.1200,AAL,2,0,16.0200
2023-02-10:17 - 22.0,5.83,0.02,22.349052,17.2000,AAL,19,0,16.0200
